<a href="https://colab.research.google.com/github/FabioRovai/Personalisation-and-Machine-Learning/blob/main/assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:

#I implemented an RNN trained on Movies classification tasks to evaluate if these reviews where good or not.
#I stored it in a .py file, keeping this notebook as clean as possible.
!wget 'https://victorzhou.com/movie-reviews-dataset.zip' -qq
!unzip -qq '/content/movie-reviews-dataset.zip' 
!pip install deepctr -qq
#move to main colab directory
%cd '/content/'
#I did a module conversion of this tutorial https://victorzhou.com/blog/keras-rnn-tutorial/ and stored it into a git.
!wget 'https://raw.githubusercontent.com/FabioRovai/test7/main/untitled49.py'
#change into correct directory, otherwse manually change train and test
%cd '/content/movie-reviews-dataset'
#run the module
%run '/content/untitled49.py'

     |████████████████████████████████| 122kB 5.2MB/s 
     |████████████████████████████████| 2.9MB 8.6MB/s 
ERROR: tensorflow 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
/content
--2021-06-07 14:43:04--  https://raw.githubusercontent.com/FabioRovai/test7/main/untitled49.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2710 (2.6K) [text/plain]
Saving to: ‘untitled49.py’

untitled49.py       100%[===================>]   2.65K  --.-KB/s    in 0s      

2021-06-07 14:43:04 (58.0 MB/s) - ‘untitled49.py’ saved [2710/2710]

/content/movie-reviews-dataset
Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'The eight Jean Rollin film I have watched is also possibly the weirdest; the intri

In [ ]:
model.evaluate(test_data)

782/782 [==============================] - 8s 10ms/step - loss: 0.5781 - accuracy: 0.7764


[0.5780869722366333, 0.7763599753379822]

In [ ]:
import pandas as pd 
import numpy as np
##Load the data 
#If raw token expires upload manually upload the dataset, is a bug or a security protocol from git.arts
df = pd.read_json("https://git.arts.ac.uk/raw/lmccallum/Personalisation/master/data/renttherunway_cleaned.json?token=AAAAAH7XGLI4G4S6DYUDL5DAYPTSO")
pd.set_option('display.max_colwidth', None)
#df[['review_text']]
#Select random result 
foo=df[['review_text']].sample()
print (foo)
print(model.predict([foo]))
foo=df[['review_text']]
#creating a new section for the review scores
df["review_score"] = model.predict([foo]) 
#print(df["review_score"].describe())
#df["rating"].describe()


                                                                                                                                                                                  review_text
98099  Very flattering fit. A tad uncomfortable but totally worth it :)  Felt like my wedding day all over again! Wore 3 inch heels in the size 4, long length (I'm 5'6) and it was perfect. 
[[0.8157796]]


In [ ]:
df.head(2)

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date,review_score
0,fit,420272,34d,2260466,137lbs,10.0,vacation,"An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",hourglass,So many compliments!,romper,"5' 8""",14,28.0,"April 20, 2016",0.813959
1,fit,273551,34b,153475,132lbs,10.0,other,"I rented this dress for a photo shoot. The theme was ""Hollywood Glam and Big Beautiful Hats"". The dress was very comfortable and easy to move around in. It is definitely on my list to rent again for another formal event.",straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36.0,"June 18, 2013",0.815930


In [ ]:
#How many interactions in total, how many unique users, how many unique items 
len(df), len(df["user_id"].unique()), len(df["item_id"].unique())

(192544, 105571, 5850)

### Dates 

Since sequence is important to us, we're going to need to sort the data by review date. Currently, ``Pandas`` sees this column as an ``object``, so we'll use ``pd.to_datetime()`` to convert the string to a date. We can then sort it, and do maths with it! 

In [ ]:
#Parse the dates
df["review_date"] = pd.to_datetime(df["review_date"])


In [ ]:
#Confirm the data types are correct
df.dtypes

fit                       object
user_id                    int64
bust size                 object
item_id                    int64
weight                    object
rating                   float64
rented for                object
review_text               object
body type                 object
review_summary            object
category                  object
height                    object
size                       int64
age                      float64
review_date       datetime64[ns]
review_score             float32
dtype: object

In [ ]:
#Sort
df = df.sort_values(by="review_date")

In [ ]:
df_copy=df.copy()

### Getting the User Sequence 

Currently, our dataset has each item on a separate row. In order to get the sequence of purchase for each user, we need to format the data. 

We also take the date in its absolute form and change it to ``days since end of dataset`` (the most recent rental being ``0 days``)

In [ ]:
users = df["user_id"].unique()

In [ ]:
data = []
last_date = df["review_date"].max()

for user in users:
    ##Get all the items for that user
    rows = df[df["user_id"]==user]
    
    #Get all the item_ids
    items = rows["item_id"]
    #Added 2 new features
    #get all the review score
    review_score = rows["review_score"]
    #get all the rating
    rating   = rows["rating"]
 
    
    #Get all the dates
    days_since_now = (last_date - rows["review_date"])
    days_since_now = np.array([i.days for i in days_since_now])
    
    #Collect into a dictionary for each user
    data.append({"item_id":items.values,"nb_days":days_since_now, "rating":rating.values, "reviews":review_score.values })

In [ ]:
#Convert to a dataframe and save
data = pd.DataFrame(data)

In [ ]:
#Made a copy
magic=data.copy()

In [ ]:
#adjusting notation
magic["reviews"] = 10 * magic["reviews"]

In [ ]:
#data
#magic
#Changed name
data=magic.copy()

In [ ]:
magic

,item_id,nb_days,rating,reviews
0,"[125564, 183200]","[2623, 1383]","[10.0, 10.0]","[8.157551, 0.37812352]"
1,"[126335, 132738, 130259]","[2520, 2096, 1745]","[8.0, 10.0, 10.0]","[8.97586, 8.940669, 7.3773484]"
2,[126335],[2511],[10.0],[7.668132]
3,"[125564, 240137, 468020]","[2510, 848, 848]","[10.0, 10.0, 10.0]","[8.123618, 8.160177, 8.107747]"
4,[190529],[2500],[10.0],[8.008156]
...,...,...,...,...
105566,[454564],[3],[8.0],[8.168954]
105567,[1498329],[3],[10.0],[8.150664]
105568,[2835159],[3],[10.0],[1.2147732]
105569,[1969604],[3],[10.0],[6.951802]


### Make the Dataset 

Now we need to make the actual **training set** that we will use for our model. 

Remember, the purpose of this model is to learn to predict the **next item rented in the sequence**. So for our input we will have 

 * [item1, item2, item3, item4, ..., itemN]
 
 * [days1, days2, days3, days4, ..., daysN]
 
 
And for our output we will have **the next item in the sequence** 
  
 
 * [itemN+1]
 
 
We also need our sequences to all be **of equal length**, and because this dataset doesn't have loads of really long sequences, we don't want to throw away stuff below a threshold! So, instead we **zero padd** the end of each sequence if its not as long as the maximum length we have picked (in this case ``5``)

* [0, 0, item1, item2, ..., itemN]
 
* [0, 0, days1, days2, ..., daysN]

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
#split into examples of 5 last things + 1, if less than 5, zero pad
training_set = []
max_len = 5
for _, user in data.iterrows():
    i = 0
    #Get all items for each user 
    items = user["item_id"]
    num_items = len(items)
    #If only one item purchased, there is no sequence! We need at least 2
    #I've found better accuracy limiting to min 3
    if num_items > 2:
        nb_days = np.array(user["nb_days"])
        #Added rating and reviews
        rating = np.array(user["rating"])
        review_score = np.array(user["reviews"])
        end = False
        #Cycle over items, taking windows of 5 and moving forwards by 1 each time
        while not end:
            
            #If we're off the end of the list, break out of the loop
            target = i + max_len
            if target >= num_items - 1:
                end = True
                target = num_items - 1
            
            #Get the input item and day features, zero padding
            input_items = pad_sequences([items[i:target]], max_len)[0]
            days = pad_sequences([nb_days[i:target]], max_len)[0]
            input_rating = pad_sequences([rating[i:target]], max_len)[0]
            input_rewiews = pad_sequences([review_score[i:target]], max_len)[0]
            
            #Get the adjusted seqeunce (shifted by one) for the target
            target_items = np.concatenate((input_items[1:],[items[target]]))
            #target_items = [items[target]]
            
            #Add to the dataset
            row = {
                "item_id":input_items,
                "nb_days":days,
                "target":target_items,
                #Added rating and reviews
                "rating":input_rating,
                "reviews":input_rewiews,
            }
            training_set.append(row)
            
            #Increment pointer
            i = i + 1

In [ ]:
training_set = pd.DataFrame(training_set)

In [ ]:
len(training_set)

34588

In [ ]:
training_set

,item_id,nb_days,target,rating,reviews
0,"[0, 0, 0, 126335, 132738]","[0, 0, 0, 2520, 2096]","[0, 0, 126335, 132738, 130259]","[0, 0, 0, 8, 10]","[0, 0, 0, 8, 8]"
1,"[0, 0, 0, 125564, 240137]","[0, 0, 0, 2510, 848]","[0, 0, 125564, 240137, 468020]","[0, 0, 0, 10, 10]","[0, 0, 0, 8, 8]"
2,"[0, 0, 0, 126335, 531077]","[0, 0, 0, 2478, 992]","[0, 0, 126335, 531077, 253667]","[0, 0, 0, 10, 10]","[0, 0, 0, 8, 8]"
3,"[0, 0, 126335, 1338469, 2261828]","[0, 0, 2474, 1105, 1105]","[0, 126335, 1338469, 2261828, 1846462]","[0, 0, 4, 8, 10]","[0, 0, 8, 8, 3]"
4,"[0, 0, 126335, 180014, 833666]","[0, 0, 2458, 1119, 735]","[0, 126335, 180014, 833666, 640839]","[0, 0, 10, 10, 10]","[0, 0, 7, 8, 7]"
...,...,...,...,...,...
34583,"[0, 0, 0, 715164, 2720289]","[0, 0, 0, 4, 4]","[0, 0, 715164, 2720289, 2665815]","[0, 0, 0, 10, 8]","[0, 0, 0, 8, 1]"
34584,"[0, 0, 0, 872442, 322704]","[0, 0, 0, 4, 4]","[0, 0, 872442, 322704, 844580]","[0, 0, 0, 8, 8]","[0, 0, 0, 8, 8]"
34585,"[0, 0, 2893615, 2072280, 2675545]","[0, 0, 4, 4, 4]","[0, 2893615, 2072280, 2675545, 1434889]","[0, 0, 8, 8, 10]","[0, 0, 1, 8, 8]"
34586,"[0, 0, 0, 1252971, 2945301]","[0, 0, 0, 4, 4]","[0, 0, 1252971, 2945301, 1793377]","[0, 0, 0, 10, 8]","[0, 0, 0, 8, 8]"


## Feature Engineering 

### Item Ids

Currently the ``item_ids`` are arbitrary and relate to the **Rent the Runway** catalogue. We are going to encode the in an **embedding** so need to make them indexes from ``0 -> num_items``

In [ ]:
#Get all the unique items from the data set
all_input_items = np.array([i for i in training_set["item_id"].values]).flatten()
all_target_items = np.array([i for i in training_set["target"].values]).flatten()
all_items = np.concatenate((all_input_items, all_target_items))
unique_items = np.unique(all_items)

In [ ]:
#Make a look up dictionary from item_id to index
item_to_index = {item_id:i for i, item_id in enumerate(unique_items)}

In [ ]:
#Swap out the new ids and convert to 2d arrays for use in training 
item_indexes = np.array([np.array([item_to_index[i] for i in ids], dtype=int) for ids in training_set["item_id"]], dtype=int)
target_indexes = np.array([np.array([item_to_index[i] for i in ids], dtype=int) for ids in training_set["target"]], dtype=int)
days = np.array([np.array([j for j in i],dtype=int) for i in training_set["nb_days"]],dtype=int)
#Added rating and reviews and converted in int.
rating = np.array([np.array([k for k in i],dtype=int) for i in training_set["rating"]],dtype=int)
reviews = np.array([np.array([l for l in i],dtype=int) for i in training_set["reviews"]],dtype=int)

### Bucketting Dates

One interesting approach they have taken at Decathalon is to **bucket** days and then **learn an embedding**, almost treating it as a **categorical variable**. 

To do this, we use ``tf.keras.layers.experimental.preprocessing.Discretization()`` to separate the days into **100 equally spaced bins**, with one more for the **zero padding** 

In [ ]:
import tensorflow as tf

In [ ]:
#Get min and max
days_min = days.min()
days_max = days.max()

#Generate 100 equally spaced boundaries 
boundaries = list(np.linspace(days_min, days_max,100,dtype=int))
boundaries.insert(0,0)
boundaries[1] = 1

#Bucket the day features
discretize_layer = tf.keras.layers.experimental.preprocessing.Discretization(
    bins=boundaries)
bucket_days = discretize_layer(days).numpy() #- 1
bucket_days = bucket_days
#Bucket the rating features
discretize_layer = tf.keras.layers.experimental.preprocessing.Discretization(
    bins=boundaries)
bucket_rat = discretize_layer(rating).numpy() #- 1
bucket_rat = bucket_rat

#Bucket the reviews features
discretize_layer = tf.keras.layers.experimental.preprocessing.Discretization(
    bins=boundaries)
bucket_rev = discretize_layer(reviews).numpy() #- 1
bucket_rev = bucket_rev

In [ ]:
days

array([[   0,    0,    0, 2520, 2096],
       [   0,    0,    0, 2510,  848],
       [   0,    0,    0, 2478,  992],
       ...,
       [   0,    0,    4,    4,    4],
       [   0,    0,    0,    4,    4],
       [   0,    0,    0,    3,    3]])

In [ ]:
bucket_days

array([[  0,   0,   0, 100,  84],
       [  0,   0,   0, 100,  35],
       [  0,   0,   0,  99,  40],
       ...,
       [  0,   0,   2,   2,   2],
       [  0,   0,   0,   2,   2],
       [  0,   0,   0,   2,   2]], dtype=int32)

## Training and Validation Sets 

Next we split the data into training and validation sets 

In [ ]:
#Generate and shuffle the indexes
total = len(training_set)
all_indexes = np.arange(total)
np.random.shuffle(all_indexes)

#Split the indexes
split = 0.9
train_indexes = all_indexes[:int(total*split)]
test_indexes = all_indexes[int(total*split):]

In [ ]:
#Make a dictionary for the inputs 
train_x = {'item_id': item_indexes[train_indexes],
          'nb_days': bucket_days[train_indexes],
           #Added rating and reviews
           'rating': bucket_rat[train_indexes],
           'reviews':bucket_rev[train_indexes]}

test_x = {'item_id': item_indexes[test_indexes],
          'nb_days': bucket_days[test_indexes],
          #Added rating and reviews
          'rating': bucket_rat[test_indexes],
          'reviews':bucket_rev[test_indexes]}

#Make an array for the outputs 
train_y = target_indexes[train_indexes]
test_y = target_indexes[test_indexes]

In [ ]:
test_y

array([[   0, 3489, 3609, 5365, 5490],
       [3148,    2, 2248, 4289, 5023],
       [5597, 4628, 5295, 4091, 4613],
       ...,
       [ 656, 1160, 1634, 3240, 1907],
       [3465, 2339, 1331,  659, 2861],
       [1016, 2968, 1516, 2234,  956]])

In [ ]:
test_x

{'item_id': array([[   0,    0, 3489, 3609, 5365],
        [   0, 3148,    2, 2248, 4289],
        [3476, 5597, 4628, 5295, 4091],
        ...,
        [1921,  656, 1160, 1634, 3240],
        [2738, 3465, 2339, 1331,  659],
        [1544, 1016, 2968, 1516, 2234]]),
 'nb_days': array([[ 0,  0, 19, 19,  4],
        [ 0, 36, 32, 30, 28],
        [11, 11, 11,  6,  6],
        ...,
        [30, 30, 29, 22, 22],
        [24, 24, 23, 16, 16],
        [22, 20, 20, 17, 14]], dtype=int32),
 'rating': array([[0, 0, 2, 2, 2],
        [0, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        ...,
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]], dtype=int32),
 'reviews': array([[0, 0, 0, 2, 2],
        [0, 0, 2, 2, 2],
        [0, 2, 2, 2, 2],
        ...,
        [0, 0, 2, 2, 2],
        [2, 2, 2, 1, 2],
        [2, 1, 2, 2, 2]], dtype=int32)}

## Build and Train the Model 

Previously we have used ``Keras's Sequential API``, where we first make a model then **sequentially add layers to it** one by one. This works because each layer only has one input, and outputs directly into the next. 

However, whilst this is broadly true of our model, its not strictly true. Our **Embedding layers** both feed into the **Concatenate layer**. So instead, we will use the [Keras Functional API](https://keras.io/guides/functional_api/).

As the documentation says 

```
The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.
```

Essentially, when we build up the ``network graph``, instead of **adding things to the model**, we **specify input layer** we want this new layer to have. 

Here, we make a layer (``x``), and then a layer(``output``), specifying that ``x`` is in the input for ``output``

```
x = layers.Dense(64, activation="relu")
outputs = layers.Dense(10)(x)
```

In [ ]:
#Model hyper parameters 
item_vocab_size = len(unique_items)
hp = {
    "embedding_item":100,
    "embedding_nb_days":20,
    #Added rating and reviews
    "embedding_rating":20,
    "embedding_reviews":20,
    "rnn_units_cat":[1024,512],
    "learning_rate":0.01
}

### ``tf.keras.Input()``

The [Input Layer](https://keras.io/api/layers/core_layers/input/) sits at the base of our ``Model`` and takes does what it says on the tin - Takes the input. 

We specfiy a dictionary of inputs to match our dictionaries we made in the **training set**. This means we have **two separate inputs**, that feed into **two separate embedding layers**.

We give them a ``batch_input_shape`` of ``[None, max_len]``, so that it know each batch will be a sequence of ``max_len`` (in our ase 5) items, but the batch_size itself isn't decided until we ``compile()`` the model. 

In [ ]:
inputs = {}
inputs['item_id'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                   name='item_id', dtype=tf.int32)

# nb_days bucketized
inputs['nb_days'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                   name='nb_days', dtype=tf.int32)
#rating
inputs['rating'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                name='rating', dtype=tf.int32)
#reviews   
inputs['reviews'] = tf.keras.Input(batch_input_shape=[None, max_len],
                                   name='reviews', dtype=tf.int32)


### ``tf.keras.layers.Embedding()``

Then we add the embedding layers, each time specifying which item from the **input dictionary** to take as input. 

The **item embedding** takes an input the size of the **number of unique items (vocab size)** and learns a mapping to a denser embedding of a given size. 

The **days embedding** takes an input the size of the **number of buckets + 1 (for zero padding)** and learns a mapping to a denser embedding of a given size. 


In [ ]:
embedding_item = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                           output_dim=hp.get('embedding_item'),
                                           name='embedding_item'
                                          )(inputs['item_id'])
# nbins=100, +1 for zero padding
embedding_nb_days = tf.keras.layers.Embedding(input_dim=100 + 1,
                                              output_dim=hp.get('embedding_nb_days'),
                                              name='embedding_nb_days'
                                             )(inputs['nb_days'])

#Added rating and reviews
embedding_rating = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                           output_dim=hp.get('embedding_rating'),
                                           name='embedding_rating'
                                          )(inputs['rating'])


embedding_reviews = tf.keras.layers.Embedding(input_dim=item_vocab_size,
                                           output_dim=hp.get('embedding_reviews'),
                                           name='embedding_reviews'
                                          )(inputs['reviews'])

In [ ]:
print(embedding_item,embedding_nb_days,embedding_rating,embedding_reviews)

KerasTensor(type_spec=TensorSpec(shape=(None, 5, 100), dtype=tf.float32, name=None), name='embedding_item/embedding_lookup/Identity_1:0', description="created by layer 'embedding_item'") KerasTensor(type_spec=TensorSpec(shape=(None, 5, 20), dtype=tf.float32, name=None), name='embedding_nb_days/embedding_lookup/Identity_1:0', description="created by layer 'embedding_nb_days'") KerasTensor(type_spec=TensorSpec(shape=(None, 5, 20), dtype=tf.float32, name=None), name='embedding_rating/embedding_lookup/Identity_1:0', description="created by layer 'embedding_rating'") KerasTensor(type_spec=TensorSpec(shape=(None, 5, 20), dtype=tf.float32, name=None), name='embedding_reviews/embedding_lookup/Identity_1:0', description="created by layer 'embedding_reviews'")


### `` tf.keras.layers.Concatenate()``

We then concatentate embedding layers into one layer 

In [ ]:
# Concatenate embedding layers
concat_embedding_input = tf.keras.layers.Concatenate(
 name='concat_embedding_input')([embedding_item, embedding_nb_days,embedding_rating, embedding_reviews  ]) 

concat_embedding_input = tf.keras.layers.BatchNormalization(
 name='batchnorm_inputs')(concat_embedding_input)

### LSTM Layers 

When then put in the ``tf.keras.layers.LSTM()`` layer, with a ``tf.keras.layers.BatchNormalization()`` either side. 

More on that in the lecture!

In [ ]:
input_layer = concat_embedding_input

for i, num_units in enumerate(hp.get('rnn_units_cat')):
    
    # LSTM layer
    rnn = tf.keras.layers.LSTM(units=num_units,
                                   return_sequences=True,
                                   recurrent_initializer='glorot_normal',
                                   name='LSTM_cat' + str(i)
                                   )(input_layer)

    rnn = tf.keras.layers.BatchNormalization(name='batchnorm_lstm' + str(i))(rnn)
    
    input_layer = rnn

# create encoding padding mask
encoding_padding_mask = tf.math.logical_not(tf.math.equal(inputs['item_id'], 0))

# Self attention so key=value in inputs
att = tf.keras.layers.Attention(use_scale=False, causal=True,
                                name='attention')(inputs=[rnn, rnn],
                                                  mask=[encoding_padding_mask,
                                                        encoding_padding_mask])



### The Output

Finally, we bring it all together in a ``tf.keras.layers.Dense()`` **softmax layer**. This means that the output of this layer will be 

``
[batch_size x max_len x item_vocab_size]
``

Where each sequence in the batch is a ``[max_len x item_vocab_size]`` tensor telling us the probability of that item in the catalogue being next in the sequence 

In [ ]:
# Last layer is a fully connected one
output = tf.keras.layers.Dense(item_vocab_size, activation = tf.nn.softmax, name='output')(att)

### Loss Function 

We write a custom loss function. This is necessary because we need to again mask out the 0s to stop the model optimising towards them. 

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
def loss_function(real, pred):
    loss = SparseCategoricalCrossentropy()(real, pred)
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    return tf.reduce_mean(loss)

### ``tf.keras.Model()``

Finally, we're ready to join the ``Inputs`` and the ``Outputs`` into a ``Model()`` object and ``compile()``.

In [ ]:
model = tf.keras.Model(inputs, output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(hp.get('learning_rate')),
    loss=loss_function,
    metrics=['sparse_categorical_accuracy'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
item_id (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
nb_days (InputLayer)            [(None, 5)]          0                                            
__________________________________________________________________________________________________
rating (InputLayer)             [(None, 5)]          0                                            
__________________________________________________________________________________________________
reviews (InputLayer)            [(None, 5)]          0                                            
______________________________________________________________________________________________

In [ ]:
history = model.fit(train_x,train_y,
                    epochs=20, 
                    verbose=1,
                    batch_size=512,
                    validation_data=(test_x, test_y))

Epoch 1/20
61/61 [==============================] - 10s 103ms/step - loss: 6.2282 - sparse_categorical_accuracy: 0.0025 - val_loss: 6.4345 - val_sparse_categorical_accuracy: 2.7962e-04
Epoch 2/20
61/61 [==============================] - 6s 91ms/step - loss: 5.5446 - sparse_categorical_accuracy: 0.0134 - val_loss: 6.0327 - val_sparse_categorical_accuracy: 0.0015
Epoch 3/20
61/61 [==============================] - 6s 91ms/step - loss: 4.7696 - sparse_categorical_accuracy: 0.0877 - val_loss: 5.3802 - val_sparse_categorical_accuracy: 0.0196
Epoch 4/20
61/61 [==============================] - 5s 90ms/step - loss: 3.8874 - sparse_categorical_accuracy: 0.2497 - val_loss: 4.6571 - val_sparse_categorical_accuracy: 0.1034
Epoch 5/20
61/61 [==============================] - 5s 90ms/step - loss: 3.1920 - sparse_categorical_accuracy: 0.3950 - val_loss: 3.8665 - val_sparse_categorical_accuracy: 0.2780
Epoch 6/20
61/61 [==============================] - 6s 91ms/step - loss: 2.7042 - sparse_categorica

## Predictions 



In [ ]:
#Predictions for whole test set
results = model.predict(test_x)

In [ ]:
#exmples x max_len x vocab size
results.shape

(3459, 5, 5701)

In [ ]:
#max_len x vocab size
results[100].shape

(5, 5701)

In [ ]:
#Get index of highest prob in item vocab
results[100][-1].sum()

1.0000001

In [ ]:
results[-1].argmax()

19337

In [ ]:
index_to_item = {v:k for k,v in item_to_index.items()}    


In [ ]:
index_to_item[70]

155381

In [ ]:
training_set.head(72)

,item_id,nb_days,target,rating,reviews
0,"[0, 0, 0, 126335, 132738]","[0, 0, 0, 2520, 2096]","[0, 0, 126335, 132738, 130259]","[0, 0, 0, 8, 10]","[0, 0, 0, 8, 8]"
1,"[0, 0, 0, 125564, 240137]","[0, 0, 0, 2510, 848]","[0, 0, 125564, 240137, 468020]","[0, 0, 0, 10, 10]","[0, 0, 0, 8, 8]"
2,"[0, 0, 0, 126335, 531077]","[0, 0, 0, 2478, 992]","[0, 0, 126335, 531077, 253667]","[0, 0, 0, 10, 10]","[0, 0, 0, 8, 8]"
3,"[0, 0, 126335, 1338469, 2261828]","[0, 0, 2474, 1105, 1105]","[0, 126335, 1338469, 2261828, 1846462]","[0, 0, 4, 8, 10]","[0, 0, 8, 8, 3]"
4,"[0, 0, 126335, 180014, 833666]","[0, 0, 2458, 1119, 735]","[0, 126335, 180014, 833666, 640839]","[0, 0, 10, 10, 10]","[0, 0, 7, 8, 7]"
...,...,...,...,...,...
67,"[0, 0, 0, 126335, 1367424]","[0, 0, 0, 2195, 1217]","[0, 0, 126335, 1367424, 423547]","[0, 0, 0, 10, 10]","[0, 0, 0, 8, 8]"
68,"[0, 0, 0, 139086, 141300]","[0, 0, 0, 2195, 1497]","[0, 0, 139086, 141300, 921642]","[0, 0, 0, 10, 10]","[0, 0, 0, 8, 8]"
69,"[146684, 2340200, 731134, 2411704, 2864831]","[2195, 678, 648, 615, 551]","[2340200, 731134, 2411704, 2864831, 349579]","[10, 8, 10, 10, 10]","[8, 0, 7, 8, 6]"
70,"[124204, 330238, 1871026, 1056174, 2686655]","[2194, 999, 605, 571, 509]","[330238, 1871026, 1056174, 2686655, 1603811]","[10, 10, 10, 8, 10]","[8, 8, 8, 8, 8]"


In [ ]:
#index to rating
all_input_rat = np.array([i for i in training_set["rating"].values]).flatten()
all_targettt_items = np.array([i for i in training_set["target"].values]).flatten()
all_items_rat = np.concatenate((all_input_rat, all_targettt_items))
unique_itemsr = np.unique(all_items_rat)
item_to_rating = {rating:i for i, rating in enumerate(unique_itemsr)}
index_to_rating = {v:k for k,v in item_to_rating.items()}  
index_to_rating[70]

152836

In [ ]:
#index to review
all_input_rev = np.array([i for i in training_set["reviews"].values]).flatten()
all_targett_items = np.array([i for i in training_set["target"].values]).flatten()
all_items_rev = np.concatenate((all_input_rev, all_targett_items))
unique_itemsrv = np.unique(all_items_rev)
item_to_review = {review:i for i, review in enumerate(unique_itemsrv)}
index_to_review = {v:k for k,v in item_to_review.items()}  
index_to_review[70]

151551

**The new features you have chosen and why**

 *The new chosen features are reviews and rating.
Rating alone was not providing a complete understanding of users feedback, where I believe that a score of 1 to 10 is not fully reliable to understand consumer satisfaction, providing possibly misleading results.
I obtained new insight on reviews, understanding if they were positive or negative. In this way, the model has a deeper understanding of users feedbacks.*

**How you have integrated them into the model**

*I used an RNN trained on movie reviews to understand if reviews were positive or not. Then I appended it as a new column. I bucketed reviews and rating and trained the model with these two new features.*

**Have they improved performance?**

*The model might be capable to provide quality to quantitative insights regarding users satisfaction.*